<a href="https://colab.research.google.com/github/krishnakalisarkar/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deliverable 1: ETL on Amazon Product Reviews.

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [510 kB]
Hit:12 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-05 02:31:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-05 02:31:06 (6.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47098686|             1|
|    9973109|             1|
|   52477231|            24|
|   27568383|             1|
|   15519001|             2|
|   52130093|             2|
|   48794123|           102|
|   12453285|             1|
|   22824019|             1|
|   21087503|             9|
|   52553295|             4|
|   44143172|             1|
|   47844321|             3|
|   40575930|             1|
|   12657201|             1|
|   38674185|             1|
|   40325486|           166|
|   42645542|             2|
|   18768108|             5|
|   10546543|           361|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
|0805087605|What's Inside You...|
|1439197334|Far from Here: A ...|
|1413748260|       Plasma Dreams|
|1613469845|           Bolt Clan|
|B005IUSVS0|       If I Were You|
|0486451550|Architectura Nava...|
|0312426267|Golden Boy: Memor...|
|0425247414|Murder on Fifth A...|
|0375722203|Random House Webs...|
|1615390588|Colodin Project, ...|
|047023847X|HTML, XHTML and C...|
|1892785390|Continuing Care S...|
|1467996149|Far Away in the S...|
|0307886948|Jeannie Out of th...|
|159038668X|The Kingdom and t...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117| 2012-05-03|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131| 2012-05-03|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025| 2012-05-03|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232| 2012-05-03|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125| 2012-05-03|
| RCLZ5OKZNUSY4|   27780192|146854456X|     270349766| 2012-05-03|
|R1S65DJYEI89G4|   13041546|1118094514|     752141158| 2012-05-03|
|R3KQYBQOLYDETV|   51692331|0563521147|     729491316| 2012-05-03|
|R3QV8K7CSU8K2W|   23108524|0669444421|     261004015| 2012-05-03|
|R3W5A1WUGO5VQ0|   51692331|1897784457|     497876045| 2012-05-03|
|R20AQCY3FMBVN5|   49438248|0316738158|     691490916| 2012-05-03|
| R7KY8VL871MVL|   11818020|0738730440|     544176812| 2012-05

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ciao1tqx9lfb.us-east-2.rds.amazonaws.com:5432/challenge_big_data_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [11]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## End of Deliverable 1

# Deliverable 2: Determine Bias of Vine Reviews.

In [ ]:
# Step 1: Total number of votes > 20
top_reviews_df = df.filter("total_votes>=20")
top_reviews_df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52534548|R18VIM840CEFRP|1621360075|     143884185|The Real Kosher J...|           Books|          1|           16|        105|   N|                N|                 SLH|I'm sorry, but ca...| 2012-05-03|
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|    

In [ ]:
# Step 2: Calculating percentage in 2 steps.
percentage_df = top_reviews_df.withColumn('percentage',top_reviews_df['helpful_votes']/top_reviews_df["total_votes"])
Percentage_df = percentage_df.withColumn("Percentage",percentage_df["percentage"]*100)
Percentage_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|        Percentage|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   52534548|R18VIM840CEFRP|1621360075|     143884185|The Real Kosher J...|           Books|          1|           16|        105|   N|                N|                 SLH|I'm sorry, but ca...| 2012-05-03|15.238095238095239|
|         US|   49541983|R14PMAJTY0EAAT|1937

In [ ]:
# Filtering percentage above 50%
high_percent_df = Percentage_df.filter("Percentage > 50")
high_percent_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       Percentage|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|95.07042253521126|
|         US|   52978967|R1363VA3TPNLVB|06700229

In [ ]:
# Step 3: Filter vine(paid)= Y
vine_paid_df = high_percent_df.filter(high_percent_df["vine"] == "Y")
vine_paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       Percentage|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|         US|   50673938|R2Z8083V8ZFQVZ|1612183549|     561835180|Redemption (The P...|           Books|          3|           82|         87|   Y|                N|     Promising debut|The positives:  t...| 2012-05-02|94.25287356321839|
|         US|   35653914| RK25TAO4GGS5G|05475673

In [ ]:
# Step 4: Filter vine(unpaid)= N
vine_unpaid_df = high_percent_df.filter(high_percent_df["vine"] == "N")
vine_unpaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       Percentage|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|95.07042253521126|
|         US|   52978967|R1363VA3TPNLVB|06700229

In [ ]:
# Step 5:  Total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for review(paid).

##Paid Reviews

In [ ]:
from pyspark.sql.functions import sum, count
# Total number of reviews
total_reviews_sum = vine_paid_df.select(sum("star_rating"))
total_reviews_sum.show()

+----------------+
|sum(star_rating)|
+----------------+
|           16929|
+----------------+



In [ ]:
# Total number of reviews
total_reviews_count = vine_paid_df.select(count("star_rating"))
total_reviews_count.show()

+------------------+
|count(star_rating)|
+------------------+
|              4743|
+------------------+



In [ ]:
# Total number of 5 star reviews
five_star_df= vine_paid_df.filter("star_rating == 5")
five_star_df.show(10)
# Total Five star reviews
total_five_star_df = five_star_df.select(sum("star_rating"))
total_five_star_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       Percentage|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------------+
|         US|   15737193|R12CHC0CB2WASU|1612183174|     198757847|           King City|           Books|          5|           43|         48|   Y|                N|There's a new she...|Detective Tom Wad...| 2012-05-02|89.58333333333334|
|         US|   47770260|R27UEUDUYPLXZC|01329446

In [ ]:
# Total Five star reviews
total_five_star_count = five_star_df.select(count("star_rating"))
total_five_star_count.show()

+------------------+
|count(star_rating)|
+------------------+
|              1604|
+------------------+



In [ ]:
# Percentage of 5 star reviews
paid_five_star_percentage = (total_five_star_count/total_reviews_count)
#round(paid_five_star_percentage, 3)
paid_five_star_percentage.show()

TypeError: ignored